# 函式 Import

In [1]:
import numpy as np

from keras.models import Sequential, load_model
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D

# Cifar 10 資料輸入

In [2]:
# X: 資料圖片 Y: one-hot label
# 分成訓練 / 測試集
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
x_train = X_train.astype('float32')/255
x_test = X_test.astype('float32')/255
# np_utils 將標籤轉成 one-hot vector
y_train = np_utils.to_categorical(Y_train)
y_test = np_utils.to_categorical(Y_test)

# 建立 Model

In [3]:
# model 的部分
model = Sequential()
# 捲積層
model.add(Conv2D(filters = 8, kernel_size = 3, input_shape = X_train.shape[1:], activation = 'relu', padding = 'same'))
# 池化層
model.add(MaxPool2D(pool_size=2))
# 攤平成一維輸入
model.add(Flatten())
# 用 Dense 當輸入層
model.add(Dense(512, activation='relu'))
# 去掉一些 node (防止overfitting.....之類的?)
model.add(Dropout(rate=0.25))
# 用 Dense 當輸出層
model.add(Dense(10, activation='softmax'))

# 訓練 Model

In [4]:
# 訓練 model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=64, verbose=1)

Epoch 1/10
782/782 [==============================] - 6s 6ms/step - loss: 1.5754 - accuracy: 0.4328
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 1.2927 - accuracy: 0.5415
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 1.1754 - accuracy: 0.5802
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 1.0926 - accuracy: 0.6117
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 1.0241 - accuracy: 0.6364
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.9559 - accuracy: 0.6650
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 0.8981 - accuracy: 0.6821
Epoch 8/10
782/782 [==============================] - 5s 6ms/step - loss: 0.8436 - accuracy: 0.7022
Epoch 9/10
782/782 [==============================] - 5s 6ms/step - loss: 0.7907 - accuracy: 0.7203
Epoch 10/10
782/782 [==============================] - 5s 6ms/step - loss: 0.7444 - accuracy: 0.7365

# 訓練結果

In [5]:
# 訓練結果
loss, accuracy = model.evaluate(x_test, y_test)
print('Test:')
print('Loss:', loss)
print('Accuracy:', accuracy)

313/313 [==============================] - 1s 4ms/step - loss: 1.0606 - accuracy: 0.6379
Test:
Loss: 1.0605803728103638
Accuracy: 0.6378999948501587


# 捲積層的 Kernel

In [6]:
# 輸出捲積層的 Kernel
model.layers[0].get_weights()[0][:, :, 0, :].transpose((2, 1, 0))

array([[[ 0.17989069,  0.03560186,  0.17690578],
        [ 0.17488232, -0.32062584, -0.32466096],
        [-0.18265557, -0.22679614, -0.20155272]],

       [[ 0.1365082 ,  0.3500096 , -0.19382577],
        [ 0.3238539 ,  0.27852535,  0.02050069],
        [-0.09544826,  0.20642783,  0.02672616]],

       [[ 0.3257345 , -0.09420446, -0.0393934 ],
        [ 0.14013724,  0.03093108, -0.13225678],
        [-0.28755552, -0.1188833 ,  0.32232234]],

       [[-0.09183131,  0.14151601,  0.10228204],
        [-0.21302095,  0.22739135, -0.1696033 ],
        [-0.03204486, -0.10643927,  0.14921153]],

       [[ 0.07323677, -0.21195613, -0.24596201],
        [-0.00985811,  0.05720398,  0.0325974 ],
        [ 0.293804  ,  0.19209564,  0.18616147]],

       [[ 0.01826923, -0.44857898, -0.09977714],
        [ 0.25623444, -0.23090765, -0.30135876],
        [ 0.1992955 ,  0.3593259 , -0.00933613]],

       [[ 0.10922879, -0.35313272,  0.11223694],
        [ 0.18135805,  0.28242996, -0.3221697 ],
        

# 運用捲積層得到的 Kernel 套用到一般圖片上
拿 3 個 kernel 出來 套用到圖片上<br>
就是...看看捲積層的 kernel 在幹嘛0.0

In [17]:
from PIL import Image
import cv2

def padding(img_name, img_w, img_h):
    img = Image.open(img_name)
    new_img = Image.new("RGB", (img_w + 2, img_h + 2), "White")
    new_img.paste(img, (1, 1))

    return cv2.cvtColor(np.asarray(new_img), cv2.COLOR_RGB2BGR)

def convolution(image_name, mask, img_w, img_h, out_name):
    out_p = padding(image_name, img_w, img_h)
    out_img = cv2.imread(image_name)        # 要輸出的圖，用 imread 只是為了取出影像格式

    # 用 padding 後的圖跟 mask 做捲積，把每個 pixel 運算結果傳給輸出影像的對應位置
    for i in range(1, img_h + 1):
        for j in range(1, img_w + 1):
          for k in range(3):
              # cur 紀錄該 pixel 的捲積結果
              cur = 0
              cur += out_p[i - 1, j - 1, k] * mask[0][0] # 左上
              cur += out_p[i - 1, j, k] * mask[0][1]     # 中上
              cur += out_p[i - 1, j + 1, k] * mask[0][2] # 右上
              cur += out_p[i, j - 1, k] * mask[1][0]     # 中左
              cur += out_p[i, j, k] * mask[1][1]         # 正中
              cur += out_p[i, j + 1, k] * mask[1][2]     # 中右
              cur += out_p[i + 1, j - 1, k] * mask[2][0] # 左下
              cur += out_p[i + 1, j, k] * mask[2][1]     # 中下
              cur += out_p[i + 1, j + 1, k] * mask[2][2] # 右下

              # 因為灰階是在 0 ~ 255 之間，超過的要把他們抓回來
              if cur > 255:
                  out_img[i - 1, j - 1, k] = 255
              elif cur < 0:
                  out_img[i - 1, j - 1, k] = 0
              else:
                  out_img[i - 1, j - 1, k] = int(cur)    # 這邊要把數值轉回整數
    # 輸出影像
    cv2.imwrite(out_name, out_img)

k1 = model.layers[0].get_weights()[0][:, :, 0, :].transpose((2, 1, 0))[1]
k2 = model.layers[0].get_weights()[0][:, :, 0, :].transpose((2, 1, 0))[2]
k4 = model.layers[0].get_weights()[0][:, :, 0, :].transpose((2, 1, 0))[4]

print("k1", k1)
print("k2", k2)
print("k4", k4)
convolution("NICO.jpg", k1, 512, 512, "out1.jpg")
convolution("NICO.jpg", k2, 512, 512, "out2.jpg")
convolution("NICO.jpg", k4, 512, 512, "out3.jpg")

k1 [[ 0.1365082   0.3500096  -0.19382577]
 [ 0.3238539   0.27852535  0.02050069]
 [-0.09544826  0.20642783  0.02672616]]
k2 [[ 0.3257345  -0.09420446 -0.0393934 ]
 [ 0.14013724  0.03093108 -0.13225678]
 [-0.28755552 -0.1188833   0.32232234]]
k4 [[ 0.07323677 -0.21195613 -0.24596201]
 [-0.00985811  0.05720398  0.0325974 ]
 [ 0.293804    0.19209564  0.18616147]]
